In [13]:
import pandas as pd
import requests
import json
import base64
import tensorflow as tf

In [14]:
data = pd.read_csv('data/data.csv')
inputs = data.iloc[53, :8].to_dict()
inputs

{'Gender': 'Male',
 'Ever_Married': 'No',
 'Age': 25,
 'Graduated': 'Yes',
 'Profession': 'Entertainment',
 'Work_Experience': 0.0,
 'Spending_Score': 'Low',
 'Family_Size': 2.0}

In [15]:
data.iloc[53, :9]

Gender                      Male
Ever_Married                  No
Age                           25
Graduated                    Yes
Profession         Entertainment
Work_Experience              0.0
Spending_Score               Low
Family_Size                  2.0
Segmentation                   1
Name: 53, dtype: object

In [16]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://sub2-production.up.railway.app/v1/models/customer-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "0" if prediction_value < 0.5 else "1"
else:
    result = "Error: No predictions found in response."

print(result)

1
